In [1]:
!pwd

/content


In [2]:
#changing the working directory and entering to dataset directory for my google drive
%cd drive/MyDrive/kaggle/image_classification/

/content/drive/MyDrive/kaggle/image_classification


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd 10_monkey_species/

/content/drive/My Drive/kaggle/image_classification/10_monkey_species


In [5]:
!ls

monkey_labels.txt  test  train


In [6]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [7]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

In [8]:
train_path = 'train'
valid_path = 'test'

In [9]:
# add preprocessing layer to the front of VGG
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58900480/58889256 [==============================] - 1s 0us/step


In [20]:
# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False

In [21]:
# useful for getting number of classes
folders = glob('train/*')

In [22]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

In [23]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [24]:
# view the structure of the model
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [25]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [26]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [27]:
training_set = train_datagen.flow_from_directory('train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1098 images belonging to 10 classes.


In [28]:
test_set = test_datagen.flow_from_directory('test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 272 images belonging to 10 classes.


In [29]:
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
35/35 [==============================] - 52s 1s/step - loss: 2.0514 - accuracy: 0.3852 - val_loss: 1.1306 - val_accuracy: 0.6176
Epoch 2/10
35/35 [==============================] - 52s 1s/step - loss: 0.7951 - accuracy: 0.7468 - val_loss: 0.7567 - val_accuracy: 0.7574
Epoch 3/10
35/35 [==============================] - 50s 1s/step - loss: 0.4706 - accuracy: 0.8634 - val_loss: 0.7115 - val_accuracy: 0.7721
Epoch 4/10
35/35 [==============================] - 51s 1s/step - loss: 0.3569 - accuracy: 0.8980 - val_loss: 0.6982 - val_accuracy: 0.7831
Epoch 5/10
35/35 [==============================] - 50s 1s/step - loss: 0.2629 - accuracy: 0.9326 - val_loss: 0.6594 - val_accuracy: 0.7868
Epoch 6/10
35/35 [==============================] - 50s 1s/step - loss: 0.2103 - accuracy: 0.9545 - val_loss: 0.6336 - val_accuracy: 0.8235
Epoch 7/10
35/35 [==============================] - 51s 1s/step - loss: 0.1534 - accuracy: 0.9699 - val_loss: 0.5895 - val_accuracy: 0.8162
Epoch 8/10
35/35 [==